# Reconstruction Tutorial - Orbit Slicing example with differing grids
This notebook shows how to use the OrbitSlicing option of the reconstruction function and how to generate reconstructed plots using different grid resolutions. Trajectories can be obtained either from the test GDC trajectory file or the flythrough trajectory functions. The GDC test trajectory was designed to be approximately stationary in GSE coordinates. See the Trajectory_Coord_Plots demo notebook for examples on flythrough trajectory options.

In [ ]:
# Get trajectory data from an old GDC trajectory file. 
from kamodo_ccmc.flythrough.Reconstruction_v0 import read_GDC_sattraj, RECON

file_dict = read_GDC_sattraj('C:/Users/rringuet/Kamodo_env/Kamodo/docs/notebooks/KGS_orbit_data.txt')
# coordinates in GDZ spherical: 'UTCtimestamps' in s, 'Longitude' in deg, 'Latitude' in deg, 'Altitude' in km
print(file_dict['UTCtimestamps'].min(), file_dict['UTCtimestamps'].max())  # min and max for later comparison to data

In [ ]:
# Find out what models are available
# The models available through the flythrough are automatically available through the reconstruction function.
from kamodo_ccmc.flythrough import model_wrapper as MW
MW.Choose_Model('')

In [ ]:
# For a given model, find out what time ranges are covered by the data in a given directory.
model = 'GITM'
file_dir = 'D:/GITM/jasoon_shim_071418_IT_1_tenth/'  #change to match your machine
times = MW.File_Times(model, file_dir)
times[0].timestamp(), times[1].timestamp()
# This function also automatically performs any data preparation needed.

In [ ]:
# Shift times in example trajectory to match model data time range.
# Reduce computation load by taking every third position
# Some of the trajectory at the end will be automatically excluded later if not covered by the data.
mask = range(0, len(file_dict['UTCtimestamps'])-1, 3)
time_diff = file_dict['UTCtimestamps'][mask].min() - 1426550400
file_dict['sat_time'] = file_dict['UTCtimestamps'][mask] - time_diff
print(file_dict['sat_time'].min(), file_dict['sat_time'].max())
# Make sure the data you choose covers at least half the time range of the trajectory.

In [ ]:
# Convert coordinates to system desired for reconstruction to take place in.
# See Trajectory_Coords_Plots notebook for more details.
coord_type, coord_grid = 'GEO','sph'  #choose a coordinate system for the reconstruction to take place in
from kamodo_ccmc.flythrough.utils import ConvertCoord
c1, c2, c3, units = ConvertCoord(file_dict['sat_time'], file_dict['Longitude'][mask], file_dict['Latitude'][mask], 
                                 file_dict['Altitude'][mask], 'GDZ', 'sph', coord_type, coord_grid)

In [ ]:
# Bring up documentation for the reconstruction functionality.
help(RECON)

In [ ]:
# User defined variables
variable_name = 'rho_n'
recon_dims = 'tc2'  # TimeLat in spherical coordinates
dx, dy = 300., 5.  # 5 minute time resolution, 5 degree latitude resolution
recon_option = 'UnMod_OrbitSliceD'  # slicing along the orbit, daytime values only, without averaging over longitude
dt = 300.  # If default dt=60 s between imaginary satellites is not desired, set dt to the desired time resolution.
# These options result in an execution time of about 2 minutes for the next block, depending on the model chosen.

In [ ]:
# In some cases, the resolution of the sampling grid for the satellite constellation should be different from
#   the desired resolution of the model data. To accomplish this, run the reconstruction without any satellite offsets
#   to retrieve the desired model data grid with a faster execution time, and then rerun the reconstruction with the 
#   desired constellation arrangement separately (see below). As originally set, this block takes about 2 minutes to 
#   complete.
recon_step1 = RECON(model,variable_name,file_dir,
              file_dict['sat_time'], c1, c2, c3, coord_type, coord_grid, recon_option, recon_dims,
              dx=dx, dy=dy, dt=dt)

In [ ]:
# Show the corresponding model data. Note the plot is produced in the coordinate system requested.
# The vertical signatures seen for reconstructions in some coordinate systems result from the drifting 
#   MLT of the satellites. This can be corrected by adding a longitudinal offset of 15 deg/hr for each 
#   additional satellite (not used here).
recon_step1.plot(rho_n_model=dict(Time=recon_step1.x, Lat=recon_step1.y))

In [ ]:
# Nicer version of same plot
fig = recon_step1.plot(rho_n_model=dict(Time=recon_step1.x, Lat=recon_step1.y))
fig.update_traces(colorscale="Viridis", ncontours=200, 
                  contours=dict(coloring="fill",showlines=False))
fig

In [ ]:
# Here, run the reconstruction function with the desired constellation arrangement and the different grid resolution.
# To decrease execution time, use the 'flythrough' option to avoid constructing a model grid using the new grid resolution.
# In this case, the constellation choice is a string of 5 satellites equally spaced in distance in the orbit.
import numpy as np
time_offsets = np.linspace(0,5548,6)[0:-1]  # equidistant = equal time offsets around orbit
dx = 5548.  # Choosing the approximate orbital period as the time resolution (see output from previous block).
# Keeping the same latitude resolution as before.
recon_step2 = RECON(model,variable_name,file_dir,
              file_dict['sat_time'], c1, c2, c3, coord_type, coord_grid, recon_option, recon_dims,
              dx=dx, dy=dy, dt=dt, time_offsets=time_offsets, run_option='flythrough')  #dx is in seconds

In [ ]:
# Show the corresponding reconstructed data. Note the differing grid resolution as compared to the previous plot.
recon_step2.plot(rho_n=dict(Time=recon_step2.x, Lat=recon_step2.y))

In [ ]:
# Nicer version of same plot
fig = recon_step2.plot(rho_n=dict(Time=recon_step2.x, Lat=recon_step2.y))
fig.update_traces(colorscale="Viridis", ncontours=200, 
                  contours=dict(coloring="fill",showlines=False))
fig

In [ ]:
# In order to compute the percent difference between the above two plots, a more logic is needed than shown in other
#   notebooks. Comparing the reconstructed data to the model data on the data's grid requires interpolating the model 
#   data onto the other grid. This can be simply done using the built-in interpolator for each grid.
# First, interpolate the model data onto the reconstructed data grid.
model_data_newgrid = recon_step1.rho_n_model(recon_step2.x, recon_step2.y).T
# Second, assign the array to the recon object with the same grid.
# syntax: create_2Dinterpolator(var_name,x_grid,y_grid,data_array,unit_string)
recon_step2.create_2Dinterpolator('rho_n_model', recon_step2.x, recon_step2.y,
                                  model_data_newgrid, recon_step2.variable_units)
# Third, calculate the percent difference and assign to the same recon object as before.
pdiff = (model_data_newgrid - recon_step2.recon_grid)/model_data_newgrid*100.
recon_step2.create_2Dinterpolator('PercentDiff', recon_step2.x, recon_step2.y,pdiff,'')
# Showing modified Kamodo object.
recon_step2  

In [ ]:
# Show model data on sparser grid.
recon_step2.plot(rho_n_model=dict(Time=recon_step2.x, Lat=recon_step2.y))

In [ ]:
# Nicer version of same plot
fig = recon_step2.plot(rho_n_model=dict(Time=recon_step2.x, Lat=recon_step2.y))
fig.update_traces(colorscale="Viridis", ncontours=200, 
                  contours=dict(coloring="fill",showlines=False))
fig

In [ ]:
# Show the percent difference between the two. A percent difference of zero is an exact match.
# How well the reconstructed plot matches the model plot not only depends on the constellation arrangement,
#  but also on the reconstruction method chosen. The 'AvgMod_...' options typically result in the better
#  matches, but are not physically representative of what the constellation will 'see' in real data because
#  the two non-reconstructed dimensions are ignored in the input satellite trajectory (e.g. an average value 
#  for both height and time are used instead of the full range for a Lon-Lat reconstruction). The unmodified 
#  options ('Unmod_...') are thus recommended as the more physical comparison because the full set of input 
#  trajectory values are used.
recon_step2.plot(PercentDiff=dict(Time=recon_step2.x, Lat=recon_step2.y))

In [ ]:
# Nicer version of same plot
fig = recon_step2.plot(PercentDiff=dict(Time=recon_step2.x, Lat=recon_step2.y))
fig.update_traces(colorscale="Viridis", ncontours=200, 
                  contours=dict(coloring="fill",showlines=False))
fig

In [ ]:
# Retrieve the percent difference data values and show in a histogram, ignoring NaN values.
# Some extra logic is required to automatically enforce bins of width 2%
import numpy as np
import matplotlib.pyplot as plt

pdiff_data = recon_step2.PercentDiff()
data_min, data_max = np.floor(np.nanmin(pdiff_data)), np.ceil(np.nanmax(pdiff_data))
num_bins = int((data_max-data_min)/2.)
if num_bins<5:
    num_bins=20
hist, edges, patches = plt.hist(np.ravel(pdiff_data), range=(data_min, data_max),bins=num_bins)
plt.xlabel('Percent Difference')
plt.ylabel('Frequency')